In [1]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools

import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
# object_detection API now accessible
from object_detection.utils import dataset_util, label_map_util 
from collections import namedtuple

# Initiate argument parser
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str)
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default=None)
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

args = parser.parse_args()

if args.image_dir is None:
    args.image_dir = args.xml_dir

label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)


def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []

  Using cached tensorflow-2.17.0-cp310-cp310-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.0-cp310-cp310-win_amd64.whl.metadata (20 kB)
  Using cached tensorboard-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.4.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached optree-0.12.1-cp310-cp310-win_amd64.whl.metadata (48 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\admin\anaconda3\envs\tensor\lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
  File "C:\Users\admin\anaconda3\envs\tensor\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\admin\anaconda3\envs\tensor\lib\ssl.py", line 1


   ----- --------------------------------- 58.1/385.0 MB 270.8 kB/s eta 0:20:08
   ----- --------------------------------- 58.1/385.0 MB 270.8 kB/s eta 0:20:08
   ----- --------------------------------- 58.1/385.0 MB 270.7 kB/s eta 0:20:08
   ----- --------------------------------- 58.2/385.0 MB 270.8 kB/s eta 0:20:07
   ----- --------------------------------- 58.2/385.0 MB 270.7 kB/s eta 0:20:08
   ----- --------------------------------- 58.2/385.0 MB 270.8 kB/s eta 0:20:07
   ----- --------------------------------- 58.2/385.0 MB 270.7 kB/s eta 0:20:08
   ----- --------------------------------- 58.3/385.0 MB 270.0 kB/s eta 0:20:11
   ----- --------------------------------- 58.3/385.0 MB 270.0 kB/s eta 0:20:11
   ----- --------------------------------- 58.3/385.0 MB 270.0 kB/s eta 0:20:11
   ----- --------------------------------- 58.3/385.0 MB 268.6 kB/s eta 0:20:17
   ----- --------------------------------- 58.3/385.0 MB 267.9 kB/s eta 0:20:20
   ----- ------------------------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\admin\anaconda3\envs\tensor\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\admin\anaconda3\envs\tensor\lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
  File "C:\Users\admin\anaconda3\envs\tensor\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\admin\anaconda3\envs\tensor\lib\ssl.py", line 1

  Using cached pycocotools-2.0.8-cp310-cp310-win_amd64.whl.metadata (1.1 kB)
  Using cached matplotlib-3.9.1-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.2.1-cp310-cp310-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     --------- ------------------------------ 41.0/165.9 kB ? eta -:--:--
     --------- ------------------------------ 41.0/165.9 kB ? eta -:--:--
     ------------------ ------------------ 81.9/165.9 kB 762.6 kB/s eta 0:00:01
     -------------------- ---------------- 92.2/165.9 kB 655.4 kB/s eta 0:00:01
     -------------------- ---------------- 92.2/165.9 kB 655.4 kB/s eta 0:00:01
     -------------------- ---------------- 92.2/165.9 kB 655.4 kB/s eta 0:00:01
     -------------------- ---------------- 92.2/165.9 kB 655.4 kB/s eta 0:00:01
     ----------

ModuleNotFoundError: No module named 'object_detection'